### PROJETO: PREVENDO DEMANDA DE ESTOQUE COM BASE EM VENDAS - EMPRESA BIMBO DO MÉXICO

### DESCRIÇÃO DOS DADOS


Descrição dos dados dos datasets utilizados neste experimento.

- Semana --> **semana**: número da semana (de quinta a quarta-feira);
- Agencia_ID --> **deposito_id**: id do depósito de vendas;
- Canal_ID --> **canal_id**:  id do canal de vendas;
- Ruta_SAK --> **rota_id**:  id da rota (várias rotas = depósito de vendas);
- Cliente_ID --> **cliente_id**: id do cliente;
- NombreCliente --> **nome_cliente**: nome do cliente;
- Producto_ID --> **produto_id**: id do produto;
- NombreProducto --> **nome_produto**: nome do produto;
- Venta_uni_hoy --> **venda_unidade**: unidade de vendas desta semana ao fornecedor (inteiro);
- Venta_hoy --> **venda_peso**: vendas esta semana (unidade: pesos);
- Dev_uni_proxima --> **retorno_unidade**: unidade de retorno na próxima semana (inteiro);
- Dev_proxima --> **devolucao_peso**: devoluções na próxima semana (unidade: pesos);
- Demanda_uni_equil --> **demanda_ajustada**: demanda ajustada (inteiro). (este é o alvo a ser previsto)

### PRÉ-PROCESSAMENTO

####  CARREGANDO BIBLIOTECAS

In [1]:
# DESATIVAR MENSAGENS DE WARNINGS (warn = -1) E ATIVAR (warn = 0)
options(warn = -1)

In [2]:
# DESATIVA NOTAÇÃO CIENTÍFICA
options(scipen = 999)

In [3]:
# PARA TAMANHO DOS GRÁFICOS
options(jupyter.plot_scale = 1)

In [4]:
# CARREGAR BIBLIOTECAS 
suppressPackageStartupMessages({
library(data.table)
library(dplyr)  
library(readr)
})

#### DADOS DE TREINO
Dataset com os dados para treino.

In [5]:
# CARREGAR DADOS DE TREINO DISPONIBILIZADOS NO KAGGLE
train_csv <- fread("train.csv", encoding = "UTF-8")

In [6]:
# RENOMEAR OS ATRIBUTOS DO DATASET DE TREINO
names(train_csv) <- c("semana", "deposito_id", "canal_venda", "rota_id", "cliente_id", "produto_id", "venda_unidade", 
                         "venda_peso", "retorno_unidade", "devolucao_peso", "demanda_ajustada")

#### COLETA DA AMOSTRA E FORMAÇÃO DA BASE DE DADOS 

Formação da base de dados a partir de amostras coletadas a partir da base de dados disponibilizada no Kaggle. Utilizou-se a função sample_n() para coletar amostras de 35.000 observações aleatórias de cada semana. No problema, existem 7 semanas. No total, a base formada para este experimento foi de 245.000 observações.

In [7]:
set.seed(1971)

In [8]:
# COLETAR AMOSTRAS DAS SEMANAS
amostras_semanas <- c(3:9)
df_bimbo <- data.frame()
for (a in amostras_semanas) {
    df_a = filter(train_csv, semana == a)
    df_a = sample_n(df_a, size = 35000)
  
    amostra = data.frame(df_a)
    df_bimbo = rbind(df_bimbo, amostra)
}
glimpse(df_bimbo)

Rows: 245,000
Columns: 11
$ semana           <int> 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,~
$ deposito_id      <int> 1311, 1618, 1347, 3221, 1126, 1595, 1232, 1631, 2014,~
$ canal_venda      <int> 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,~
$ rota_id          <int> 1251, 1601, 2053, 1618, 1024, 6602, 1004, 1256, 1213,~
$ cliente_id       <int> 150962, 4100032, 134877, 4588484, 1279340, 2388568, 7~
$ produto_id       <int> 1146, 31719, 43147, 40217, 1150, 44371, 1160, 1309, 1~
$ venda_unidade    <int> 1, 2, 3, 1, 2, 4, 8, 1, 3, 1, 1, 1, 14, 4, 11, 3, 2, ~
$ venda_peso       <dbl> 21.39, 15.18, 13.62, 8.98, 27.92, 29.92, 150.88, 6.76~
$ retorno_unidade  <int> 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,~
$ devolucao_peso   <dbl> 0.00, 7.59, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,~
$ demanda_ajustada <int> 1, 1, 3, 1, 2, 4, 8, 1, 3, 1, 1, 1, 14, 4, 11, 3, 2, ~


In [9]:
# 1 - VERIFICAR SE O DATASET POSSUI VALORES NA
any(is.na(df_bimbo))

[1] FALSE

In [10]:
# 2 - EXISTE VALOR MENOR QUE ZERO EM Retorno.Unidade e Demanda.Ajustada?
any(df_bimbo$retorno_unidade < 0)
any(df_bimbo$demanda_ajustada < 0)

[1] FALSE

[1] FALSE

In [11]:
# 3 - SALVANDO ARQUIVO 
write_csv(df_bimbo,'df_bimbo.csv')

####  DADOS PRODUTOS
Dataset com os nomes do produtos.

In [12]:
# CARREGAR DATASET
producto_tabla_csv <- fread("producto_tabla.csv", encoding = "UTF-8")
glimpse(producto_tabla_csv)

Rows: 2,592
Columns: 2
$ Producto_ID    <int> 0, 9, 41, 53, 72, 73, 98, 99, 100, 106, 107, 108, 109, ~
$ NombreProducto <chr> "NO IDENTIFICADO 0", "Capuccino Moka 750g NES 9", "Bimb~


In [13]:
# RENOMEAR COLUNAS DO DATSET
names(producto_tabla_csv) <- c("produto_id", "nome_produto")

In [14]:
# FUNÇÃO PARA REMOVER ACENTOS E SUBSTITUIR MINÚSCULAS POR MAIÚSCULAS 
substitui_acentos_minusculas <- function(coluna){
    string_com_acento_minusculas <- "áàãâäÀÁÃÂÄéèêëÉÈÊËíìïÍÌÏóòõôöÓÒÕÔÖúùüÚÙÜñÑabcdefghijklmnopqrstuvwxyz,"
    string_sem_acento_maiusculas <- "AAAAAAAAAAEEEEEEEEIIIIIIOOOOOOOOOOUUUUUUNNABCDEFGHIJKLMNOPQRSTUVWXYZ "
    string_final <- chartr(
        old = string_com_acento_minusculas,
        new = string_sem_acento_maiusculas,
        x = coluna)
}

In [15]:
# REMOVER ACENTOS
producto_tabla_csv[,2] <- substitui_acentos_minusculas(producto_tabla_csv$nome_produto)

In [16]:
head(producto_tabla_csv)

produto_id,nome_produto
0,NO IDENTIFICADO 0
9,CAPUCCINO MOKA 750G NES 9
41,BIMBOLLOS EXT SAJONJOLI 6P 480G BIM 41
53,BURRITOS SINCRO 170G CU LON 53
72,DIV TIRA MINI DORADITA 4P 45G TR 72
73,PAN MULTIGRANO LINAZA 540G BIM 73


In [17]:
write_csv(producto_tabla_csv, "df_produto.csv")

####  DADOS CLIENTES
Dataset com os nomes dos clientes.

In [18]:
# CARREGANDO O DATASET
cliente_tabla_csv <- fread("cliente_tabla.csv", encoding = "UTF-8")
glimpse(cliente_tabla_csv)

Rows: 935,362
Columns: 2
$ Cliente_ID    <int> 0, 1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,~
$ NombreCliente <chr> "SIN NOMBRE", "OXXO XINANTECATL", "SIN NOMBRE", "EL MORE~


In [19]:
# RENOMEANDO O DATSET
names(cliente_tabla_csv) <- c("cliente_id", "nome_cliente")

In [20]:
# REMOVENDO ACENTOS
cliente_tabla_csv[,2] <- substitui_acentos_minusculas(cliente_tabla_csv$nome_cliente)

In [21]:
head(cliente_tabla_csv)

cliente_id,nome_cliente
0,SIN NOMBRE
1,OXXO XINANTECATL
2,SIN NOMBRE
3,EL MORENO
4,SDN SER DE ALIM CUERPO SA CIA DE INT
4,SDN SER DE ALIM CUERPO SA CIA DE INT


In [22]:
write_csv(cliente_tabla_csv, "df_cliente.csv")

#### DADOS ESTADOS
Dataset com os nomes das cidades e estados.

In [23]:
# CARREGANDO O DATASET
town_state_csv <- fread("town_state.csv", encoding = "UTF-8")
glimpse(town_state_csv)

Rows: 790
Columns: 3
$ Agencia_ID <int> 1110, 1111, 1112, 1113, 1114, 1116, 1117, 1118, 1119, 1120,~
$ Town       <chr> "2008 AG. LAGO FILT", "2002 AG. AZCAPOTZALCO", "2004 AG. CU~
$ State      <chr> "MÉXICO, D.F.", "MÉXICO, D.F.", "ESTADO DE MÉXICO", "MÉXICO~


In [24]:
# RENOMEANDO O DATSET
names(town_state_csv) <- c("deposito_id", "cidade", "estado")

In [25]:
# REMOVENDO ACENTOS E PADRÕES
town_state_csv[,2] <- substitui_acentos_minusculas(town_state_csv$cidade)
town_state_csv[,3] <- substitui_acentos_minusculas(town_state_csv$estado)

In [26]:
head(town_state_csv)

deposito_id,cidade,estado
1110,2008 AG. LAGO FILT,MEXICO D.F.
1111,2002 AG. AZCAPOTZALCO,MEXICO D.F.
1112,2004 AG. CUAUTITLAN,ESTADO DE MEXICO
1113,2008 AG. LAGO FILT,MEXICO D.F.
1114,2029 AG.IZTAPALAPA 2,MEXICO D.F.
1116,2011 AG. SAN ANTONIO,MEXICO D.F.


In [27]:
glimpse(town_state_csv)

Rows: 790
Columns: 3
$ deposito_id <int> 1110, 1111, 1112, 1113, 1114, 1116, 1117, 1118, 1119, 1120~
$ cidade      <chr> "2008 AG. LAGO FILT", "2002 AG. AZCAPOTZALCO", "2004 AG. C~
$ estado      <chr> "MEXICO  D.F.", "MEXICO  D.F.", "ESTADO DE MEXICO", "MEXIC~


In [28]:
write_csv(town_state_csv, "df_estado.csv")

Nas etapas acima foram realizadas:

- Carregamento do dataset original train.csv do Kaggle;
- Coleta de amostras de dados de cada semana. Foram coletados 35.000 observações de cada semana a partir do dataset train.csv;
- Tratamento dos dados nos datasets de produto (producto_tabla.csv), clientes (cliente_tabla.csv) e estados (town_state.csv);
- Por fim, a formação do dataset (df_bimbo.csv) utilizado no experimento.